# SVD baseline algorithm using surprise package:

using `matrix_factorization.SVD` algorithm from http://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

---
### [KBV09] Yehuda Koren. Matrix factorization techniques for recommender systems.

see [Matrix Factorization techniques](https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf)

explicit feedback = (user,item, rating) represented as user-item matrix.

Matrix factorization models map both users and items to a joint latent factor space of dimensionality f, such that user-item interactions are modeled as inner products in that space.

Assume $r_{ui} = q_i^Tp_u$, how to compute $q_i, p_u$?

Want to minimize $min_{p_u, q_i} \sum_{x_{ui}}(r_{ui}-p_u^Tq_i)^2 + \lambda ($$ \lVert q_i \rVert $$^2 + $$ \lVert p_u \rVert $$^2)$

1. Can use SGD to optimize (see Simon Funk) <- focus here
2. Can use ALS (convexifies the objective)

__Adding Biases__:
some users tend to give higher/lower ratings then others. And some items tend to receive higher/lower ratings than others (relatively seen).

Bias involved in rating $r_{ui}$ is denoted by $b_{ui}$:<br/>
$b_{ui} = \mu + b_i + b_u$ <br/>
$\mu$: average rating over all movies <br/>
$b_i$: deviation of item i from average <br/>
$b_u$: deviation of user u from average <br/>

estimate of rating is: <br/>
$r_{ui} = \mu + b_i + b_u + q_i^Tp_u$

adjusted objective: <br/>
 $min_{p_u, q_i} \sum_{x_{ui}}(r_{ui}-\mu-b_u-b_i -p_u^Tq_i)^2 + \lambda ($$ \lVert q_i \rVert $$^2 + $$ \lVert p_u \rVert $$^2 + b_u^2 + b_i^2)$

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import helpers
from surprise_helpers import CustomReader, get_ratings_from_predictions
from surprise import Reader, Dataset
from surprise.model_selection.search import RandomizedSearchCV
from surprise.prediction_algorithms.matrix_factorization import SVD

## Data loading

In [2]:
reader = CustomReader()
filepath = helpers.get_train_file_path()
data = Dataset.load_from_file(filepath, reader=reader)

## Search over params


In [ ]:
param_grid = {'n_epochs': stats.randint(5,20), 
              'lr_all': stats.uniform(0.002,0.005),
              'reg_all': stats.uniform(0.02,0.6),
              'n_factors': stats.randint(50,150),
             }      
        

gs = RandomizedSearchCV(algo_class=SVD, param_distributions=param_grid, measures=['rmse'], 
                        cv=10, joblib_verbose=100, n_jobs=-1, n_iter=100)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
pd.DataFrame.from_dict(gs.cv_results)

## Results: params 

note: run on Leonhard cluster (20 cores and 22GB mem)
cv=10

0.98489843593
{'lr_bi': 0.0043688979424132048, 'lr_bu': 0.005613464182452876, 'lr_pu': 0.022148955075059311, 'lr_qi': 0.012530626289208299, 'n_factors': 151, 'reg_bi': 0.042342905803236859, 'reg_bu': 0.073415041823889998, 'reg_pu': 0.12513844856777867, 'reg_qi': 0.060228735604464054}

0.984953534195
{'lr_bi': 0.0040080904244357771, 'lr_bu': 0.0083752718099860367, 'lr_pu': 0.010813216008005243, 'lr_qi': 0.018992188100229335, 'n_factors': 158, 'reg_bi': 0.02384386240305832, 'reg_bu': 0.087957272220067495, 'reg_pu': 0.17370665665508969, 'reg_qi': 0.042977558957037719}

0.9853091859177997
{'lr_bi': 0.0050565201838808, 'lr_bu': 0.005529901817482, 'lr_pu': 0.021009781711363945, 'lr_qi': 0.019237920706921686, 'n_factors': 153, 'reg_bi': 0.027507293418947705, 'reg_bu': 0.04792231303919493, 'reg_pu': 0.1051659555261933, 'reg_qi': 0.05739791764348274}

0.985423383238
{'lr_bi': 0.0045872175952940859, 'lr_bu': 0.0059198576112051132, 'lr_pu': 0.03996299827445985, 'lr_qi': 0.012744793911701359, 'n_factors': 156, 'reg_bi': 0.039006956026578499, 'reg_bu': 0.058835165460011773, 'reg_pu': 0.065999449644116956, 'reg_qi': 0.11943136846519176}

0.98576635754
{'lr_bi': 0.004388787077144584, 'lr_bu': 0.0077070689204476878, 'lr_pu': 0.037207361664974598, 'lr_qi': 0.011526606878613951, 'n_factors': 155, 'reg_bi': 0.055491394820528769, 'reg_bu': 0.025510174666992846, 'reg_pu': 0.073005652506841268, 'reg_qi': 0.084369830119022798}

0.985920828862
{'lr_bi': 0.0061067123779277675, 'lr_bu': 0.005972391928230663, 'lr_pu': 0.023802281518036983, 'lr_qi': 0.016117532546202691, 'n_factors': 157, 'reg_bi': 0.013775300394299965, 'reg_bu': 0.034301808303089626, 'reg_pu': 0.09627118462747386, 'reg_qi': 0.061245371260615522}

0.986355638337
{'lr_bi': 0.0053921528073649578, 'lr_bu': 0.014983628487378223, 'lr_pu': 0.025070307435290061, 'lr_qi': 0.0073965320227803123, 'n_factors': 154, 'reg_bi': 0.027365169456015866, 'reg_bu': 0.03585534141987403, 'reg_pu': 0.15019357711020351, 'reg_qi': 0.050166823853083298}

0.986851655568
{'lr_bi': 0.0053647824382532638, 'lr_bu': 0.0052769550628727633, 'lr_pu': 0.012545946659482476, 'lr_qi': 0.012795461232426109, 'n_factors': 156, 'reg_bi': 0.056418600300998704, 'reg_bu': 0.022556567389521144, 'reg_pu': 0.061086848339968164, 'reg_qi': 0.077168605512323629}

0.9872175653005135
{'lr_bi': 0.006088703690766571, 'lr_bu': 0.012744554096322995, 'lr_pu': 0.012380073174962966, 'lr_qi': 0.009130873179651982, 'n_factors': 158, 'reg_bi': 0.012919509150189782, 'reg_bu': 0.036795609500178635, 'reg_pu': 0.10361453028270178, 'reg_qi': 0.048785998289016745}

0.988058743973
{'lr_bi': 0.0045634712043475644, 'lr_bu': 0.01064562075669161, 'lr_pu': 0.012470219046795966, 'lr_qi': 0.0069239309430058314, 'n_factors': 160, 'reg_bi': 0.038064835563680136, 'reg_bu': 0.083204917488220814, 'reg_pu': 0.044129727640422105, 'reg_qi': 0.083532754108250978}

0.9882588317180945
{'lr_bi': 0.005572132295112126, 'lr_bu': 0.010459976354655289, 'lr_pu': 0.012621031473530591, 'lr_qi': 0.009189138000229718, 'n_factors': 81, 'reg_bi': 0.09508515857935072, 'reg_bu': 0.08100593615106191, 'reg_pu': 0.08560885565821767, 'reg_qi': 0.06136778252645429}

0.9889400786056195
{'lr_bi': 0.00505521532063321, 'lr_bu': 0.012181115620042492, 'lr_pu': 0.0094679873549249, 'lr_qi': 0.00612337262019204, 'n_factors': 108, 'reg_bi': 0.023237810934149287, 'reg_bu': 0.09931980200620626, 'reg_pu': 0.07865295413390713, 'reg_qi': 0.04964170302513018}

0.9894911251394463
{'lr_bi': 0.006439713022658003, 'lr_bu': 0.005220730820427025, 'lr_pu': 0.012121360196274768, 'lr_qi': 0.010296116770652414, 'n_factors': 98, 'reg_bi': 0.07326008897853639, 'reg_bu': 0.0654521219387276, 'reg_pu': 0.07673672028564435, 'reg_qi': 0.07732908168371602}

-----

0.992159817137
{'lr_all': 0.0088540357639887227, 'n_factors': 94, 'reg_all': 0.060980144283391415}

0.9922037152942101
{'lr_all': 0.009404839806784696, 'n_factors': 83, 'reg_all': 0.06361145065359476}

0.9923411994233705
{'lr_all': 0.009518436883800755, 'n_factors': 98, 'reg_all': 0.06448898038350545}

0.996356760492
{'lr_all': 0.0069735838056049675, 'n_factors': 16, 'reg_all': 0.040538641816816053}


0.996617863993
{'lr_all': 0.0080655611939959484, 'n_epochs': 19, 'n_factors': 9, 'reg_all': 0.042201220509606799}

0.996918766638
{'lr_all': 0.010438622204618025, 'n_factors': 16, 'reg_all': 0.069969144045048129}

-----

0.998777808857
{'n_factors': 5}

0.999721712849
{'lr_all': 0.0048483945439183485, 'n_factors': 9}

1.00085021593
{'lr_all': 0.0035314408264436933, 'n_epochs': 19, 'n_factors': 50, 'reg_all': 0.027105037999075404}

1.00534587695
{'lr_all': 0.0034656840329879137, 'n_epochs': 10, 'n_factors': 42, 'reg_all': 0.12231592623013628}

1.00104676332
{'lr_all': 0.0066032381482039656, 'n_epochs': 17, 'n_factors': 107, 'reg_all': 0.036362623151074552}

1.00382744957
{'lr_all': 0.0045664408289589759, 'n_epochs': 12, 'n_factors': 9, 'reg_all': 0.04029560227746723}

## Train

In [3]:
# choose optimal params from above
# {'lr_bi': 0.0043688979424132048, 'lr_bu': 0.005613464182452876, 'lr_pu': 0.022148955075059311, 'lr_qi': 0.012530626289208299, 'n_factors': 151, 'reg_bi': 0.042342905803236859, 'reg_bu': 0.073415041823889998, 'reg_pu': 0.12513844856777867, 'reg_qi': 0.060228735604464054}
algo = SVD(n_factors=151, lr_bi=0.0043688979424132048, lr_bu=0.005613464182452876, lr_pu=0.022148955075059311, lr_qi=0.012530626289208299, reg_bi=0.042342905803236859, reg_bu=0.073415041823889998, reg_pu=0.12513844856777867, reg_qi=0.060228735604464054)

# train 
algo.fit(data.build_full_trainset())

## Predicting
We load the test data to predict.

In [4]:
test_file_path = helpers.get_test_file_path()
test_data = Dataset.load_from_file(test_file_path, reader=reader)
testset = test_data.construct_testset(test_data.raw_ratings)
predictions = algo.test(testset)
predictions[0]

Prediction(uid=36, iid=0, r_ui=3.0, est=3.332676908676297, details={'was_impossible': False})

We need to convert the predictions into the right format.

In [5]:
ratings = get_ratings_from_predictions(predictions)

Now we can write the file.

In [6]:
output = helpers.write_submission(ratings, 'submission_surprise_SVD_3.csv')
print(output[0:100])

Id,Prediction
r37_c1,3.332677
r73_c1,3.129160
r156_c1,3.778833
r160_c1,3.352926
r248_c1,3.466300
r25
